<a href="https://colab.research.google.com/github/edoardochiarotti/class_datascience/blob/main/2024/07_Endogeneity/07_Panel_data.ipynb"
   target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Panel Data

<img src="https://i.imgflip.com/84gdes.jpg" width="500">

In [61]:
# PACKAGES
%matplotlib inline
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
import statistics as st
import pandas as pd
import os
import statsmodels.api as sm
import re
!pip install stargazer
from stargazer.stargazer import Stargazer
!pip install linearmodels
from linearmodels.panel import PanelOLS # conda install -c conda-forge linearmodels

# FUNCTIONS FROM PACKAGES
from numpy.linalg import inv
from sklearn.linear_model import LinearRegression

# SEABORN THEME
scale = 0.4
W = 16*scale
H = 9*scale
sns.set(rc = {'figure.figsize':(W,H)})
sns.set_style("white")

- In this class we'll cover panel data and fixed effects, with the usual mix of theory and application in Python.
- This class is based on several **sources**. The main ones are:
    - Class notes by Nicolas Berman and Daniele Rinaldo at The Geneva Graduate Institute.
    - Chapter [Panel Data and Fixed Effects](https://matheusfacure.github.io/python-causality-handbook/14-Panel-Data-and-Fixed-Effects.html) from [Causal Inference for the Brave and the True](https://matheusfacure.github.io/python-causality-handbook/landing-page.html) by Matheus Facure Alves.

## Outline
- [Panel Data: Definitions and Quality of Governments Panel Database](#Panel-Data:-Definitions-and-Quality-of-Governments-Panel-Database)
- [Fixed Effects Model](#Fixed-Effects-Model)

## Panel Data: Definitions and Quality of Governments Panel Database <a name="Panel-Data:-Definitions-and-Quality-of-Governments-Panel-Database"></a>
- A large strand of applied econometrics deal with cross-sections of countries, households, or firms, which are followed over time. This type of data is called panel (or longitudinal) data.
- **Definition of panel data**: a longitudinal, or panel, dataset is one that follows a given sample of individuals over time, and thus provides multiple observations on each individuals of the sample. (Hsiao (2003), "Analysis of Panel data", second edition, p.2).
- Note: The second dimension of panel data is typically time, and we call it "time" here, but it needs not to be time.
- We call a **balanced panel** a dataset for which the number of time observations is the same for all countries. We will assume in this lecture that we have a balanced panel. However, in real life in most cases we will have access to unbalanced panels. This will be due to missing observations. This is usually not a problem; the theory discussed here applies to **unbalanced panels** as well, as long as missing observations are not missing in an "endogenous" way, in the sense that the fact that they are missing does not depend on the dependent variable. We will not discuss sample selection. So let's assume that the sample is balanced or that selection is "exogenous".
- Let's get the **QoG** database:

In [65]:
# get data
link = "https://www.qogdata.pol.gu.se/data/qog_ei_sept21.xlsx"
df_qog = pd.read_excel(link)

In [66]:
# get variables
indexes = ["ccodealp","year"]
variabs_co2 = ["edgar_co2gdp","edgar_co2t","edgar_co2pc"]
variabs_control = ["oecd_cctr_gdp"]
variabs = variabs_co2 + variabs_control
df = df_qog.loc[:,np.append(indexes,variabs)]

# make gdp per capita
df["gdp"] = (df["edgar_co2gdp"]/df["edgar_co2t"])**(-1) # billions US dollars
df["pop"] = (df["edgar_co2pc"]/df["edgar_co2t"])**(-1) # millions
df["gdp_pc"] = df["gdp"]/df["pop"] # thousands of US dollars
variabs = np.append(variabs, ["gdp","pop","gdp_pc"])

# put ones into data
df["ones"] = 1

# drop nas
df = df.dropna()

In [67]:
# describe
df.describe()

,year,edgar_co2gdp,edgar_co2t,edgar_co2pc,oecd_cctr_gdp,gdp,pop,gdp_pc,ones
count,1968.000000,1968.000000,1968.000000,1968.000000,1968.000000,1968.000000,1968.000000,1968.000000,1968.0
mean,2006.558943,0.243792,244.822353,5.525416,76.901269,696.540916,44.117529,21.766931,1.0
std,6.878167,0.172754,953.697678,5.314153,31.627199,2085.847978,166.648815,19.054821,0.0
min,1994.000000,0.017499,0.103620,0.035062,0.000000,0.997115,0.090686,0.715969,1.0
25%,2001.000000,0.139422,4.879796,1.103488,72.921500,32.436325,3.451545,6.670379,1.0
50%,2007.000000,0.204393,24.277523,4.147764,90.376500,109.278612,9.883835,16.290105,1.0
75%,2012.000000,0.278301,103.328379,8.730894,99.383250,417.710330,24.991825,32.941329,1.0
max,2018.000000,1.154208,10671.339219,29.154586,108.098000,19062.770270,1397.030000,117.517686,1.0


In [68]:
# n of countries by year
df.groupby(['year']).size().reset_index(name='counts')

,year,counts
0,1994,57
1,1995,64
2,1996,65
3,1997,65
4,1998,68
5,1999,68
6,2000,76
7,2001,78
8,2002,78
9,2003,78


In [69]:
# from 2010 to 2016
df = df.loc[(df["year"] >= 2010) & (df["year"] <= 2017),:]
df.groupby(['year']).size().reset_index(name='counts')

,year,counts
0,2010,90
1,2011,89
2,2012,89
3,2013,90
4,2014,91
5,2015,86
6,2016,85
7,2017,82


In [70]:
# plot
sns.scatterplot(x='gdp_pc', y='edgar_co2pc', data=df, color = "r", s = 20)

<Axes: xlabel='gdp_pc', ylabel='edgar_co2pc'>

In [71]:
# drop outliers quick and dirty
df = df.loc[(df["gdp_pc"] < 65) & (df["edgar_co2pc"] < 20),:]
df.groupby(['year']).size().reset_index(name='counts')

,year,counts
0,2010,87
1,2011,86
2,2012,86
3,2013,87
4,2014,88
5,2015,82
6,2016,81
7,2017,78


In [72]:
# lets make it balanced for simplicity
df_agg = df.groupby(['year']).size().reset_index(name='counts')
year_min = df_agg.loc[df_agg.counts == min(df_agg.counts),"year"].values
ccodealp_sub = np.unique(df.loc[df.year == int(year_min),"ccodealp"].values)
df = df.loc[np.isin(df.ccodealp, ccodealp_sub),:]

df_agg = df.groupby(['ccodealp']).size().reset_index(name='counts')
cou_drop = df_agg.loc[df_agg.counts == min(np.unique(df_agg["counts"])),"ccodealp"].values
df = df.loc[~np.isin(df.ccodealp, cou_drop),:]

df.groupby(['year']).size().reset_index(name='counts')

/var/folders/50/z62d1r092mvbd7y3wgvjp3jc0000gp/T/ipykernel_51396/3323309517.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ccodealp_sub = np.unique(df.loc[df.year == int(year_min),"ccodealp"].values)


,year,counts
0,2010,77
1,2011,77
2,2012,77
3,2013,77
4,2014,77
5,2015,77
6,2016,77
7,2017,77


In [73]:
# plot
sns.scatterplot(x='gdp_pc', y='edgar_co2pc', data=df, color = "r", s = 20)

<Axes: xlabel='gdp_pc', ylabel='edgar_co2pc'>

In [74]:
# maybe logs?
df["ln_gdp_pc"] = np.log(df["gdp_pc"])
df["ln_edgar_co2pc"] = np.log(df["edgar_co2pc"])

# plot
sns.scatterplot(x='ln_gdp_pc', y='ln_edgar_co2pc', data=df, color = "r", s = 20)

<Axes: xlabel='gdp_pc', ylabel='edgar_co2pc'>

## Fixed Effects Model <a name="Fixed-Effects-Model"></a>
- During last class, we have explored whether higher income leads to higher emissions, possibly through higher consumption. We have used a cross-sectional sample of 94 countries to estimate that, on average, if GDP per capita increases by 1%, we'd expect CO2 emissions per capita to increase by around 1.3245 %. This estimate comes from a specification that includes also a measure for carbon-tax revenues, as we believe that this variable can be a source of omitted-variable bias.
- Now can our estimate still be biased? Maybe. For example, let's think about how **cultural traits** can lead people to generally pollute less. We could make the case that some countries have a culture of extending the life of products and re-use, at least more than other countries. This aspect may come from the way people in these countries prepare food, their religious beliefs, etc. As a result, people in these countries consume less in general, and therefore they may consume also less carbon-intensive goods and services. If these countries with such a cultural trait are also generally poorer (plausible), then we might be overestimating the impact of income on emissions. Indeed, we might be seeing a huge difference in CO2 emissions between poor and rich countries mainly because rich countries have no culture for recycling and re-using. So it is not because I am rich that I pollute more, but it is because of my culture. If I were poor, I'd still pollute a lot.
- So how can we disentangle the impact of income per se and culture on emissions? Cultural aspects are generally unobservable, so we can't simply find a variable (observable) that controls for them. What we can do is using panel data and include countries' fixed effects.
- Let us re-write the model for the data generating process with 2 regressors in a **panel-data setup with fixed effects**, for $i=1,...,N$ and $t=1,...,T$:
<br><br>
$$
y_{it} = \beta_0 + \beta_1 x_{1,it} + \beta_2 x_{2,it} + \delta u_i + \epsilon_{it}
$$
<br>
- The new term $u_i$ refers to country-specific unobservable characteristics that might be related with $x_{2,it}$ and also affect $y_{it}$, such as the cultural aspects mentioned above. This new term is what is usually refer to as fixed effects. The important point is that we cannot observe them, so we can't just control for them by including them in our regression, as we did for $x_{2,it}$. So how do we do?
- The trick is that, rather than having 1 variable for $u_i$ with data on a specific (unobservable) aspect, we can include N **country dummies** which equal 1 for a given country and zero otherwise, and which capture any specific characteristic of a specific country.

Conceptually, for $i=1,...,N$ and $t=1,...,T$:
<br><br>
$$
y_{it} = \beta_0 + \beta_1 x_{1,it} + \beta_2 x_{2,it} + \delta_1 u_{1,i} + ... + \delta_N u_{N,i} + \epsilon_{it}
$$
<br>
- Let's write this model in expanded data-matrix notation, so we get a sense of what these dummies look like:
<br><br>
$$
\begin{bmatrix}
y_{11} \\
\vdots \\
y_{1T} \\
- \\
\vdots \\
- \\
y_{N1} \\
\vdots \\
y_{NT}
\end{bmatrix} = 
\beta_0
\begin{bmatrix}
1 \\
\vdots \\
1 \\
- \\
\vdots \\
- \\
1 \\
\vdots \\
1
\end{bmatrix} +
\beta_1
\begin{bmatrix}
x_{1,11} \\
\vdots \\
x_{1,1T} \\
- \\
\vdots \\
- \\
x_{1,N1} \\
\vdots \\
x_{1,NT}
\end{bmatrix} +
\beta_2
\begin{bmatrix}
x_{2,11} \\
\vdots \\
x_{2,1T} \\
- \\
\vdots \\
- \\
x_{2,N1} \\
\vdots \\
x_{2,NT}
\end{bmatrix} +
\delta_1
\begin{bmatrix}
1 \\
\vdots \\
1 \\
- \\
\vdots \\
- \\
0 \\
\vdots \\
0
\end{bmatrix} + ... +
\delta_N
\begin{bmatrix}
0 \\
\vdots \\
0 \\
- \\
\vdots \\
- \\
1 \\
\vdots \\
1
\end{bmatrix} + 
\begin{bmatrix}
\epsilon_{11} \\
\vdots \\
\epsilon_{1T} \\
- \\
\vdots \\
- \\
\epsilon_{N1} \\
\vdots \\
\epsilon_{NT}
\end{bmatrix}
$$
<br>

- Or in a more compact way:
<br><br>
$$
\begin{bmatrix}
y_{11} \\
\vdots \\
y_{1T} \\
- \\
\vdots \\
- \\
y_{N1} \\
\vdots \\
y_{NT}
\end{bmatrix} = 
\begin{bmatrix}
1 & x_{1,11} & x_{2,11} \\
\vdots & \vdots & \vdots \\
1 & x_{1,1T} & x_{2,1T} \\
- & - & - \\
\vdots & \vdots & \vdots \\
- & - & - \\
1 & x_{1,N1} & x_{2,N1} \\
\vdots \\
1 & x_{1,NT} & x_{2,NT}
\end{bmatrix}
\begin{bmatrix}
\beta_0 \\
\beta_1 \\
\beta_2
\end{bmatrix}
+
\begin{bmatrix}
1 & \dots & 0 \\
\vdots & \vdots & \vdots \\
1 & \dots & 0 \\
- & & - \\
\vdots & \vdots & \vdots \\
- & & - \\
0 & \dots & 1 \\
\vdots \\
0 & \dots & 1 \\
\end{bmatrix}
\begin{bmatrix}
\delta_1 \\
\vdots \\
\delta_N
\end{bmatrix} + 
\begin{bmatrix}
\epsilon_{11} \\
\vdots \\
\epsilon_{1T} \\
- \\
\vdots \\
- \\
\epsilon_{N1} \\
\vdots \\
\epsilon_{NT}
\end{bmatrix}
$$
<br>

- Or in standard matrix notation:
<br>
$$
\boldsymbol{y} = \boldsymbol{X}\boldsymbol{\beta} + \boldsymbol{U}\boldsymbol{\delta} + \boldsymbol{\epsilon}  \qquad \text{with } \boldsymbol{\epsilon} | \boldsymbol{X} \sim \mathcal{N}(\boldsymbol{0}_{NT},\,\sigma^2\boldsymbol{I}_{NT})
$$
<br>
- Aright. So the dummies capture any country specific characteristic. If we look at the expanded matrixes, it becomes clear what the $\delta$ coefficients are. As you remember well, we saw that the coefficient related to a vector of ones, i.e. $\beta_0$, is the mean of $y$ when the other covariates equal zero (intercept). In a similar way, $\delta_1$ is the mean of $y$ for country 1 when the other covariates equal zero. Hold this thought, we'll use it in a second.
- So cool, if we want to estimate this panel-data model, we can simply add these dummies, estimate the $\delta$ coefficients, and we'd have controlled for any unobserved country-specific source of endogeneity. Right? Yes and no. We can definitely do it, and we will, just if we do it we might have a computational issue. Think if we have to do this for like 250 countries, or 1 million individuals. We'd need to include 250 country dummies, so our data matrixes would be huge, and inverting huge matrixes is **computationally expensive**. Is there another way of doing an OLS estimation, in which we avoid inverting huge matrixes? It turns out that there is, and the first step to understand this methodology is starting from the **Frisch-Waugh-Lovell partitioned regression theorem**. 

<img src="https://i.imgflip.com/84gw9w.jpg" width="400">

- In a nutshell, this theorem says that if we split our regressors in 2 matrixes, we can obtain the OLS coefficients related to the first matrix with the following steps: (i) regress $y$ on the second matrix, (ii) regress the first  matrix on the second matrix, (iii) compute the residuals of (i) and the residuals of (ii), (vi) regress the residuals obtained in (i) on the residuals obtained in (ii), and the coefficient estimates of this last step will be the coefficient estimates related to the first matrix. If we apply this to our panel-data model, we can think of $X$ as the first matrix and $U$ as the second matrix. So in our case these steps would be:
    1. Estimate $\hat{\boldsymbol{y}}=\boldsymbol{U}\hat{\boldsymbol{\delta}}_{1,OLS}^*$
    2. Estimate $\hat{\boldsymbol{X}}=\boldsymbol{U}\hat{\boldsymbol{\delta}}_{2,OLS}^*$
    3. Get residuals of first regression $\tilde{\boldsymbol{y}}$ and of second regression $\tilde{\boldsymbol{X}}$
    4. Estimate $\tilde{\boldsymbol{y}} = \tilde{\boldsymbol{X}}\hat{\boldsymbol{\beta}}_{OLS}$
<br><br>
- OK so that's the principle. Now it comes the trick. In the first step what we are doing is regressing $\boldsymbol{y}$ only on country dummies. The coefficient estimates of these country dummies in this regression represent the country-level sample means of $\boldsymbol{y}$. Remember that the mean of $\boldsymbol{y}$ is the coefficient estimate related a vector of ones (mean model). So the mean of $\boldsymbol{y}$ in country 1 will be the coefficient estimate related to a vector that equals 1 for country 1 and 0 otherwise. The mean of $\boldsymbol{y}$ in country 2 will be the sum of the coefficient estimate for the first country and the coefficient estimate related to a vector that equals 1 for country 2 and 0 otherwise.
- Let's do an **example** in which we run a regression with only **year dummies** as regressors (instead of country dummies, but it's the same principle). In Python, it can be done in the following way:

In [80]:
# dummy reg on years
est_ols = sm.OLS.from_formula("ln_edgar_co2pc ~ C(year)", data=df).fit()
est_ols.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.7153,0.163,4.385,0.000,0.395,1.036
C(year)[T.2011],0.0138,0.231,0.060,0.952,-0.439,0.467
C(year)[T.2012],0.0522,0.231,0.226,0.821,-0.401,0.505
C(year)[T.2013],0.0576,0.231,0.250,0.803,-0.395,0.511
C(year)[T.2014],0.0451,0.231,0.195,0.845,-0.408,0.498
C(year)[T.2015],0.0374,0.231,0.162,0.871,-0.416,0.490
C(year)[T.2016],0.0387,0.231,0.168,0.867,-0.414,0.492
C(year)[T.2017],0.0276,0.231,0.120,0.905,-0.425,0.481


- So the mean of `ln_edgar_co2pc` for year 2010 (automatically assigned as the first year) is 0.7153. Then, the mean of `ln_edgar_co2pc` for year 2011 is 0.7153 + 0.0138 = 0.7291. Then, the mean of `ln_edgar_co2pc` for year 2011 is 0.7153 + 0.0522 = 0.7675. Etc.
- Let's now compute the mean by year in the standard way:

In [82]:
# compute mean by year
df.groupby("year")["ln_edgar_co2pc"].mean()

year
2010    0.715323
2011    0.729167
2012    0.767532
2013    0.772939
2014    0.760421
2015    0.752701
2016    0.753989
2017    0.742919
Name: ln_edgar_co2pc, dtype: float64

- Yes the mean of `ln_edgar_co2pc` for 2010 is 0.7153, for 2011 is 0.7291, for 2012 is 0.7675, etc.
- OK so goign **back to our steps** and our country dummies. So to estimate the coefficients in our **step (i)**, we do not have to do it through the usual OLS regression approach - which would imply inverting the dummy matrix (computationally expensive) - but we can simply obtain these estimates by computing the country-specific means for `ln_edgar_co2pc`. The same holds for **step (ii)**, so we just need to compute the country-specific means for `ln_gdp_pc` and `oecd_cctr_gdp`. Let's do it:

In [84]:
# compute means by country
y = "ln_edgar_co2pc"
X = ["ln_gdp_pc","oecd_cctr_gdp"]
mean_df = df.groupby("ccodealp")[[y] + X].mean()
mean_df.head()

,ln_edgar_co2pc,ln_gdp_pc,oecd_cctr_gdp
ccodealp,,,
ARG,1.567531,3.164668,99.738500
AUT,2.110968,3.972746,97.008500
BEL,2.237120,3.889788,93.023750
BFA,-1.854761,0.636703,99.996375
BGR,1.887667,2.941232,87.595000


- OK so now we have to compute the residuals. Let's store our means in vectors $\overline{\boldsymbol{y}}$ and  $\overline{\boldsymbol{X}}$ that look like this:
<br><br>
$$
\overline{\boldsymbol{y}} = 
\begin{bmatrix}
\overline{y}_{1} \\
\vdots \\
\overline{y}_{1} \\
- \\
\vdots \\
- \\
\overline{y}_{N} \\
\vdots \\
\overline{y}_{N} \\
\end{bmatrix}
\qquad
\qquad
\overline{\boldsymbol{X}} = 
\begin{bmatrix}
\overline{x}_{1,1} & \overline{x}_{2,1} \\
\vdots & \vdots \\
\overline{x}_{1,1} & \overline{x}_{2,1} \\
- & - \\
\vdots & \vdots \\
- & - \\
\overline{x}_{1,N} & \overline{x}_{2,N} \\
\vdots & \vdots \\
\overline{x}_{1,N} & \overline{x}_{2,N} \\
\end{bmatrix}
$$
<br>
- So we can obtain our residuals for steps (i) and (ii) in the following way:
    - $\tilde{\boldsymbol{y}} = \boldsymbol{y} - \overline{\boldsymbol{y}}$
    - $\tilde{\boldsymbol{X}} = \boldsymbol{X} - \overline{\boldsymbol{X}}$
- Note that for this to hold, our $\boldsymbol{U}$ and $\boldsymbol{X}$ matrixes are not exactly as the ones we have said above, as the vector of ones for the intercept is not contained in $\boldsymbol{X}$ but is contained in $\boldsymbol{U}$ and substitutes the first country. Pas grave (no big deal).
- So let's do **step (iii)** compute the residuals for step (i) and (ii) in this clever way in Python. To do that, we need to set the indexes of our data as the country iso codes:

In [86]:
# compute demeaned df
demeaned_df = (df
               .set_index("ccodealp") # set the index as the person indicator
               [[y] + X]
               - mean_df) # subtract the mean data

demeaned_df.head()

,ln_edgar_co2pc,ln_gdp_pc,oecd_cctr_gdp
ccodealp,,,
ARG,-0.031291,-0.017373,0.1225
ARG,-0.000048,0.030484,0.1605
ARG,0.018326,0.009665,-0.2025
ARG,-0.009530,0.022960,-0.1915
ARG,0.018718,-0.012814,0.0505


- Finally, **step (iv)**, let's regress the residuals from step (i) on the residuals of step (ii). Formally we estimate:
<br><br>
$$
\tilde{\boldsymbol{y}} = \tilde{\boldsymbol{X}}\boldsymbol{\beta} + \tilde{\boldsymbol{\epsilon}}
$$
<br>
- Another way of saying that we have controlled for country dummies without actually running the regressions, is saying that we have "wiped out" the all unobserved (and observed) variables that are constant across time. To see that, just re-write the model in demeaned terms (we do this without the matrix notation for simplicity). For $i=1,...,N$ and $t=1,...,T$:
<br><br>
$$
(y_{it} - \overline{y}_{it}) = (\beta_0 - \beta_0) + \beta_1 (x_{1,it} -  \overline{x}_{1,it}) + \beta_2 (x_{2,it} -  \overline{x}_{2,it}) + \delta (u_i - u_i) + (\epsilon_{it} - \overline{\epsilon}_{it})
$$
<br>
- So let's do the regression of the residuals from step (i) on the residuals from step (ii) to obtain our  estimates for $\beta$, which we'll call fixed effect estimates obtained with the FE estimator $\hat{\boldsymbol{\beta}}_{FE}$ (as they are as if we were controlling for all these country dummies):

In [88]:
# compute with ols fe estimator
est_fe = sm.OLS.from_formula(f"{y} ~ {'+'.join(X)}", data=demeaned_df).fit()
est_fe.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-2.714e-18,0.005,-5.33e-16,1.000,-0.010,0.010
ln_gdp_pc,0.2710,0.078,3.476,0.001,0.118,0.424
oecd_cctr_gdp,0.0008,0.001,0.791,0.429,-0.001,0.003


- The estimate for `ln_gdp_pc` is much lower than before! This means that, indeed, it is possible that much of the effect that we were measuring before was driven by country-specific characteristics, and not by the average effect of income on emissions. Note that the result is still statistically significant. Cool.
- VERY IMPORTANT POINT: the interpretation of the coefficient does not change and it remains the usual one.
- As we love matrix notation and our own functions so much, let's do the same by hand. If we want to keep using the matrix notation we used for the OLS, we need to come up with a matrix that demeans the data. Let's define $\boldsymbol{\iota}_T$ as a T-dimensional column vector of ones. Let us then define the following $(NT\times NT)$ matrix $\boldsymbol{P}$ that computes the means of data matrixes and stores them in the stacked way we saw above:
<br><br>
$$
\boldsymbol{P}=\boldsymbol{I}_N \otimes \frac{1}{T}\boldsymbol{\iota}_T\boldsymbol{\iota}_T'
$$
<br>
- Let's then use this matrix to compute a $(NT\times NT)$ matrix $\boldsymbol{Q}$ that de-means the data:
<br><br>
$$
\boldsymbol{Q}=\boldsymbol{I}_{NT} - \boldsymbol{P}
$$
<br>
- OK so our demeaned model can be written as:
<br><br>
$$
\boldsymbol{Q}\boldsymbol{y} = \boldsymbol{Q}\boldsymbol{X}\boldsymbol{\beta} + \boldsymbol{Q}\boldsymbol{\epsilon}
$$
<br>
- Which, as by construction $\tilde{\boldsymbol{y}}=\boldsymbol{Q}\boldsymbol{y}$, $\tilde{\boldsymbol{X}}=\boldsymbol{Q}\boldsymbol{X}$, and $\tilde{\boldsymbol{\epsilon}} = \boldsymbol{Q}\boldsymbol{\epsilon}$, is the model we wrote before:
<br><br>
$$
\tilde{\boldsymbol{y}} = \tilde{\boldsymbol{X}}\boldsymbol{\beta} + \tilde{\boldsymbol{\epsilon}}
$$
<br>
- In the exercises we'll ask you to use these equations to write your own function for the fixed effect estimator. In real life, you do not need to do that, as Python already is using these equations in a canned routine called `PanelOLS`:

In [90]:
# Canned method
est_fe_canned = PanelOLS.from_formula("ln_edgar_co2pc ~ ln_gdp_pc + oecd_cctr_gdp + EntityEffects",
                            data=df.set_index(["ccodealp", "year"]))
result = est_fe_canned.fit()
result.summary.tables[1]

,Parameter,Std. Err.,T-stat,P-value,Lower CI,Upper CI
ln_gdp_pc,0.2710,0.0833,3.2530,0.0012,0.1073,0.4346
oecd_cctr_gdp,0.0008,0.0010,0.7400,0.4597,-0.0012,0.0027


- One thing we never covered in class is **heteroscedasticity**. This is a huge topic. For now let's just say that you always need to use robust standard errors, both in cross-sectional and panel-data analysis. In panel data, the most common way to have robust standard errors is clustering them, usually at the country level (or individual level). Let's do it with the canned method:

In [92]:
# canned method with SE clustered at the country level
est_fe_canned = PanelOLS.from_formula("ln_edgar_co2pc ~ ln_gdp_pc + oecd_cctr_gdp + EntityEffects",
                            data=df.set_index(["ccodealp", "year"]))

result = est_fe_canned.fit(cov_type='clustered', cluster_entity=True)
result.summary.tables[1]

,Parameter,Std. Err.,T-stat,P-value,Lower CI,Upper CI
ln_gdp_pc,0.2710,0.1984,1.3660,0.1725,-0.1187,0.6607
oecd_cctr_gdp,0.0008,0.0013,0.5859,0.5582,-0.0018,0.0033


- WOAH! When we use robust standard errors (clustered at the country level) the coefficient estimate for `ln_gdp_pc` is no longer statistically significant at the 10% level!! This is huge. So basically we are saying that, after we control for country-specific unobservables, GDP per capita is no longer useful to predict CO2 emissions per capita. So it's all about country-specific unonservables, that can be culture and other stuff!! That's mind blowing.
- To see the size of the bias, you can estimate the panel-data model without taking into account for the fixed effects. Let's do it with the canned function:

In [94]:
# compute with ols pooled
est_pool_canned = PanelOLS.from_formula("ln_edgar_co2pc ~ ln_gdp_pc + oecd_cctr_gdp",
                            data=df.set_index(["ccodealp", "year"]))

result = est_pool_canned.fit(cov_type='clustered', cluster_entity=True)
result.summary.tables[1]

,Parameter,Std. Err.,T-stat,P-value,Lower CI,Upper CI
ln_gdp_pc,0.8453,0.0843,10.023,0.0000,0.6797,1.0109
oecd_cctr_gdp,-0.0154,0.0029,-5.3853,0.0000,-0.0211,-0.0098


- Finally, to understand better FE, let's try to visualize them.
- Let's subset our data:

In [96]:
# subset
df_plot = df[(df["ln_gdp_pc"] > 2) & (df["ln_gdp_pc"] < 3)].copy()

# estimate ols and store fitted values
est_ols = sm.OLS.from_formula("ln_edgar_co2pc ~ ln_gdp_pc", data=df_plot).fit()
df_plot["fitted_values"] = est_ols.fittedvalues

# plot observations
sns.scatterplot(x='ln_gdp_pc', y='ln_edgar_co2pc', data=df_plot, color = "r", s = 20)

# plot ols fit line (crossing fitted values)
sns.lineplot(x='ln_gdp_pc', y='fitted_values', data=df_plot)

<Axes: xlabel='gdp_pc', ylabel='edgar_co2pc'>

In [97]:
# subset
df_plot = df[(df["ln_gdp_pc"] > 2) & (df["ln_gdp_pc"] < 3)].copy()

# get number of countries
cou = np.unique(df_plot.loc[:,"ccodealp"].values)
colors = sns.color_palette(n_colors=len(cou)).as_hex()

# estimate ols and store fitted values
est_ols = sm.OLS.from_formula("ln_edgar_co2pc ~ ln_gdp_pc", data=df_plot).fit()
df_plot["fitted_values"] = est_ols.fittedvalues

# estimate fe with dummies and store fitted values
est_fe = sm.OLS.from_formula("ln_edgar_co2pc ~ ln_gdp_pc + C(ccodealp)", data=df).fit()
df_plot["fitted_values_fe"] = est_fe.fittedvalues

# plot observations
sns.scatterplot(x='ln_gdp_pc', y='ln_edgar_co2pc', data=df_plot, hue="ccodealp", legend = False, 
                palette = colors)

# plot fe fit lines (crossing fitted values)
for i in range(len(cou)):
    df_temp = df_plot.query(f"ccodealp=='{cou[i]}'")
    sns.lineplot(x='ln_gdp_pc', y='fitted_values_fe', data=df_temp, hue="ccodealp", legend = False)

- Take a minute to appreciate what the image above is telling you about what fixed effect is doing. Notice that fixed effect is fitting one regression line per country. Also notice that the lines are parallel. The slope of the line is the effect of GDP per capita on CO2 emissions per capita. So the fixed effect is assuming that the causal effect is constant across all countries.
- Something that we did not cover is time fixed effects, and two-way fixed effects, which can be estimated with the canned function.